In [1]:
import numpy as np 
import pandas as pd
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D,MaxPool2D,ZeroPadding2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import vgg16
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns

import os
import cv2
import numpy as np
from tensorflow.keras.models import model_from_json
from tensorflow.keras.utils import img_to_array

In [2]:
df = pd.read_csv('/kaggle/input/fer2013/fer2013.csv') 
df.head(5)

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [3]:
emotions = {
    0:"Angry",
    1:"Disgust",
    2:"Fear",
    3:"Happy",
    4:"Sad",
    5:"Surprize",
    6:"Neutral"
}

In [4]:
count_usage = (df.groupby(df['Usage'])['pixels'].count()).to_frame() # to_frame is used to convert a pandas Series to a dataframe
count_usage

,pixels
Usage,
PrivateTest,3589
PublicTest,3589
Training,28709


In [5]:
X_train,y_train = [],[]
X_val,y_val = [],[]
X_test,y_test = [],[]
for index,row in df.iterrows():
    k = row['pixels'].split(" ")
    if row['Usage'] == 'Training':
        X_train.append(np.array(k))
        y_train.append(row['emotion'])
    elif row['Usage'] == 'PrivateTest':
        X_test.append(np.array(k))
        y_test.append(row['emotion'])
    elif row['Usage'] == 'PublicTest':
        X_val.append(np.array(k))
        y_val.append(row['emotion'])

X_train = np.array(X_train,dtype='float')
y_train = np.array(y_train)
X_test = np.array(X_test,dtype='float')
y_test = np.array(y_test)
X_val = np.array(X_val,dtype='float')
y_val = np.array(y_val)

In [6]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape,X_val.shape,y_val.shape

((28709, 2304), (28709,), (3589, 2304), (3589,), (3589, 2304), (3589,))

In [7]:
X_train = X_train.reshape(X_train.shape[0],48,48,1)
X_test = X_test.reshape(X_test.shape[0],48,48,1)
X_val = X_val.reshape(X_val.shape[0],48,48,1)

y_train = to_categorical(y_train,num_classes=7) 
y_test = to_categorical(y_test,num_classes=7)
y_val = to_categorical(y_val,num_classes=7)

X_train.shape,y_train.shape,X_test.shape,y_test.shape,X_val.shape,y_val.shape

((28709, 48, 48, 1),
 (28709, 7),
 (3589, 48, 48, 1),
 (3589, 7),
 (3589, 48, 48, 1),
 (3589, 7))

In [8]:
def cnn_model():
    model = Sequential()

    model.add(ZeroPadding2D((1,1),input_shape=(48,48,1))) # input shape of the VGG-16 model is typically set to (224, 224, 3) or (227, 227, 3) for standard image datasets.
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1))) # it allows to apply padding to images
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPool2D((2,2), strides=(2,2))) 

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPool2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPool2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPool2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPool2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='softmax'))

    return model

In [9]:
model = cnn_model()
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_pad1 (ZeroPadding2D)  (None, 50, 50, 1)         0         
_________________________________________________________________
block1_l1 (Conv2D)           (None, 48, 48, 64)        640       
_________________________________________________________________
block1_pad2 (ZeroPadding2D)  (None, 50, 50, 64)        0         
_________________________________________________________________
block1_l2 (Conv2D)           (None, 48, 48, 64)        36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 24, 24, 64)        0         
_________________________________________________________________
block2_pad1 (ZeroPadding2D)  (None, 26, 26, 64)        0         
_________________________________________________________________
block2_l1 (Conv2D)           (None, 24, 24, 128)       7

In [10]:
checkpointer = [EarlyStopping(monitor = 'val_accuracy', verbose = 1, 
                              restore_best_weights=True,mode="max",patience = 10),
                              ModelCheckpoint('best_model.h5',monitor="val_accuracy",verbose=1,
                              save_best_only=True,mode="max")]

history = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=30,
    callbacks=checkpointer,
    validation_data=(X_val,y_val)
)

Epoch 1/30
898/898 [==============================] - 27s 27ms/step - loss: 1.8239 - accuracy: 0.2466 - val_loss: 1.8071 - val_accuracy: 0.2494

Epoch 00001: val_accuracy improved from -inf to 0.24937, saving model to best_model.h5
Epoch 2/30
898/898 [==============================] - 24s 27ms/step - loss: 1.7538 - accuracy: 0.2826 - val_loss: 1.6875 - val_accuracy: 0.3455

Epoch 00002: val_accuracy improved from 0.24937 to 0.34550, saving model to best_model.h5
Epoch 3/30
898/898 [==============================] - 24s 27ms/step - loss: 1.5844 - accuracy: 0.3804 - val_loss: 1.5806 - val_accuracy: 0.3840

Epoch 00003: val_accuracy improved from 0.34550 to 0.38395, saving model to best_model.h5
Epoch 4/30
898/898 [==============================] - 24s 27ms/step - loss: 1.4511 - accuracy: 0.4308 - val_loss: 1.5328 - val_accuracy: 0.4043

Epoch 00004: val_accuracy improved from 0.38395 to 0.40429, saving model to best_model.h5
Epoch 5/30
898/898 [==============================] - 24s 27ms/

In [17]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
